In [1]:
import h5py
import tensorflow as tf

In [2]:
class train_generator:
    def __call__(self):
      with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(int(len(hf['X'])*0.8)): # both hf1 and hf2 have same length
                yield hf['X'][i], hf['y'][i]
                yield hf2['X'][i], hf2['y'][i]                

In [3]:
with h5py.File('data/electron.hdf5', 'r') as hf:
  test_ds_start=int(len(hf['X'])*0.8)+1
test_ds_start

199201

In [4]:
class test_generator:
    def __call__(self):
      with h5py.File('data/electron.hdf5','r') as hf2:
        with h5py.File('data/photon.hdf5', 'r') as hf:
            for i in range(int(len(hf['X'])*0.2)): # both hf1 and hf2 have same length
                yield hf['X'][test_ds_start+i], hf['y'][test_ds_start+i]
                yield hf2['X'][test_ds_start+i], hf2['y'][test_ds_start+i]  

In [5]:
train_ds = tf.data.Dataset.from_generator(
  train_generator(),
  output_signature=(
    tf.TensorSpec(shape=(32, 32, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
  )
)
test_ds = tf.data.Dataset.from_generator(
  test_generator(),
  output_signature=(
    tf.TensorSpec(shape=(32, 32, 2), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
  )
)

In [6]:
iter_train_ds=iter(train_ds)
def input_fn():
    feature, label = next(iter_train_ds)
    return {"x": feature}, label


In [7]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.FlatMapDataset

In [8]:
batched_train_ds=train_ds.batch(32)
batched_test_ds=test_ds.batch(32)

In [9]:
iter_batched_train_ds=iter(batched_train_ds)
iter_batched_test_ds=iter(batched_test_ds)
def input_fn_batched():
    feature, label = next(iter_batched_train_ds)
    return {"x": feature}, label
def test_fn_batched():
  feature, label = next(iter_batched_test_ds)
  return {"x": feature}, label

In [10]:
feature, label = next(iter_train_ds)
tf.reduce_max(feature)

<tf.Tensor: shape=(), dtype=float32, numpy=0.90156156>

In [21]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',input_shape=(32, 32, 2)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),


    tf.keras.layers.Flatten(),
   
    tf.keras.layers.Dense(256, activation='relu'),
   
  
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 30, 30, 16)        304       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 15, 15, 16)       0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 15, 15, 16)        0         
                                                                 
 conv2d_15 (Conv2D)          (None, 13, 13, 32)        4640      
                                                                 
 conv2d_16 (Conv2D)          (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 5, 5, 64)         0         
 g2D)                                                 

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy',tf.keras.metrics.AUC()])

In [ ]:
model.fit(
  batched_train_ds,
  validation_data=batched_test_ds,
  epochs=3
)

Epoch 1/3
    797/Unknown - 3036s 4s/step - loss: 0.6667 - accuracy: 0.5945 - auc: 0.6257

In [23]:
model.save_weights('./best.h5')